# Vorbereitung des Datensatzes

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta

print("Setup complete")

Setup complete


## 1. Hauptdaten vorbereiten

In [2]:
main_data = pd.read_csv('../input/2023-12-20/data_austin_15min.csv')

# Umrechnung des Zeitstempels in UTC, da verschiedene Zeitzonen +
# Entfernen der Zeitzone und Umwandlung in naives datetime-Objekt
main_data['timestamp'] = pd.to_datetime(main_data.local_15min, utc=True).dt.tz_localize(None)

# Gesamtverbrauch des Hauses berechnen
main_data['use'] = main_data[['grid', 'solar']].sum(axis=1, skipna=True)

# Entfernen unnötiger Spalten
columns_to_drop = ['local_15min', 'grid', 'solar', 'leg1v', 'leg2v']
main_data.drop(columns=columns_to_drop, inplace=True)

main_data.sample(5)

,dataid,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,...,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,timestamp,use
568778,7901,-0.001,NaN,NaN,NaN,NaN,NaN,NaN,0.018,0.029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-21 21:45:00,1.151
676369,8386,1.074,NaN,NaN,NaN,NaN,NaN,NaN,0.036,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18 07:30:00,1.618
779372,9160,0.009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-04-27 20:15:00,0.149
529451,7800,0.002,NaN,NaN,NaN,NaN,NaN,NaN,0.003,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2018-03-06 17:00:00,0.596
681573,8386,0.221,NaN,NaN,NaN,NaN,NaN,NaN,0.025,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-11 12:30:00,0.690


## 2. Innentempteratur vorbereiten

In [3]:
# Daten aus mehreren Dateien verbinden
folder_path = '../input/2023-12-20/indoor_temp'
temp_dfs = list()

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    temp_dfs.append(df)

indoor_temp_data = pd.concat(temp_dfs, ignore_index=True)

# Datentypen ändern
indoor_temp_data['timestamp'] = pd.to_datetime(indoor_temp_data.localminute)

# Auswahl der relevanten Spalten
relevant_columns = ['dataid', 'timestamp', 'temp_c']
indoor_temp_data = indoor_temp_data[relevant_columns]

# Spalten umbenennen
indoor_temp_data.rename(columns={'temp_c': 'indoor_temp'}, inplace=True)

indoor_temp_data.sample(5)

,dataid,timestamp,indoor_temp
87417220,1283,2014-09-17 05:05:01,24.00
77727228,6412,2014-09-30 13:19:02,23.31
50629320,9643,2014-09-19 18:39:01,23.38
25718216,484,2014-10-17 21:04:01,28.25
35791947,6412,2014-10-01 09:44:01,23.31


## 3. Wetterdaten vorbereiten

In [4]:
def convert_fahrenheit_to_celsius(fahrenheit):
    """
    Funktion zur Umrechnung von Fahrenheit zu Celsius.
    :param fahrenheit: Temperatur in Grad Fahrenheit
    :return: Temperatur in Grad Celsius
    """
    celsius = round((fahrenheit - 32) * 5 / 9, 2)
    return celsius

In [5]:
weather_data = pd.read_csv('../input/2023-12-20/weather.csv')

# Daten für Stadt Austin (Texas, USA) filtern
weather_data = weather_data[(weather_data.latitude == 30.292432) & (weather_data.longitude == -97.699662)]

# Umrechnung von Fahrenheit zu Celsius
weather_data['outdoor_temp'] = weather_data.temperature.apply(convert_fahrenheit_to_celsius)
weather_data['app_outdoor_temp'] = weather_data.apparent_temperature.apply(convert_fahrenheit_to_celsius)

# Datentypen ändern
weather_data['timestamp'] = pd.to_datetime(weather_data.localhour)

# Auswahl der relevanten Spalten
relevant_columns = ['timestamp', 'outdoor_temp', 'app_outdoor_temp', 'humidity']
weather_data = weather_data[relevant_columns]

weather_data.sample(5)

,timestamp,outdoor_temp,app_outdoor_temp,humidity
179642,2019-12-15 03:00:00,10.14,10.14,0.92
59894,2013-10-30 17:00:00,27.60,30.26,0.73
145575,2016-08-13 03:00:00,26.35,26.35,0.81
170155,2018-11-15 09:00:00,6.17,4.89,0.62
758,2011-02-01 09:00:00,-1.54,-8.01,0.76


## 4. Daten zu Feiertagen und Wochenenden vorbereiten
* https://www.austintexas.gov/department/official-city-holidays
* https://chat.openai.com/c/a8b42663-3daf-419d-87e5-f2beea0ef92b

In [6]:
def get_holidays(year):
    """
    Erstellt ein Dictionary der festen und beweglichen Feiertage für ein gegebenes Jahr in den USA.

    :param year: Das Jahr, für das die Feiertage berechnet werden sollen. Es sollte ein ganzzahliges Jahr sein (z.B. 2022).

    :return: Ein Dictionary, in dem die Schlüssel die Namen der Feiertage sind und die Werte die entsprechenden Datumsangaben.
             Die Funktion berücksichtigt spezielle Regeln für bewegliche Feiertage sowie Anpassungen für Feiertage, 
             die auf das Wochenende fallen.

    Die berechneten Feiertage umfassen:
    - Neujahrstag
    - Martin-Luther-King-Tag (dritter Montag im Januar)
    - Memorial Day (letzter Montag im Mai)
    - Juneteenth (19. Juni)
    - Unabhängigkeitstag (4. Juli)
    - Tag der Arbeit (erster Montag im September)
    - Veteranentag (11. November)
    - Erntedankfest (vierter Donnerstag im November)
    - Thanksgiving-Freitag (Tag nach Thanksgiving)
    - Heiligabend (24. Dezember)
    - Weihnachtstag (25. Dezember)
    
    Anmerkung: Für Feiertage, die auf das Wochenende fallen, werden Anpassungen vorgenommen, sodass der Feiertag auf den
    nächstgelegenen Wochentag verschoben wird.
    """
    holidays = {
        "Neujahrstag": date(year, 1, 1),
        "Martin-Luther-King-Tag": date(year, 1, 1) + timedelta(days=(14 - date(year, 1, 1).weekday() + 7) % 7),
        "Memorial Day": date(year, 5, 31) - timedelta(days=date(year, 5, 31).weekday()),
        "Juneteenth": date(year, 6, 19),
        "Unabhängigkeitstag": date(year, 7, 4),
        "Tag der Arbeit": date(year, 9, 1) + timedelta(days=(7 - date(year, 9, 1).weekday()) % 7),
        "Veteranentag": date(year, 11, 11),
        "Erntedankfest": date(year, 11, 1) + timedelta(days=(24 + (3 - date(year, 11, 1).weekday() + 7) % 7)),
        "Thanksgiving-Freitag": date(year, 11, 1) + timedelta(days=(25 + (3 - date(year, 11, 1).weekday() + 7) % 7)),
        "Heiligabend": date(year, 12, 24),
        "Weihnachtstag": date(year, 12, 25)
    }
    # Anpassung für Feiertage, die auf das Wochenende fallen
    if holidays["Neujahrstag"].weekday() == 6:  # Sonntag
        holidays["Neujahrstag"] = date(year, 1, 2)
    if holidays["Veteranentag"].weekday() in [5, 6]:  # Samstag oder Sonntag
        holidays["Veteranentag"] = date(year, 11, 10) if holidays["Veteranentag"].weekday() == 5 else date(year, 11, 12)
    if holidays["Juneteenth"].weekday() in [5, 6]:  # Samstag oder Sonntag
        holidays["Juneteenth"] = date(year, 6, 18) if holidays["Juneteenth"].weekday() == 5 else date(year, 6, 20)
    
    return holidays

# Erstellen einer Liste aller Tage von 2017 bis 2020
start_date = date(2017, 1, 1)
end_date = date(2020, 12, 31)
delta = end_date - start_date

dates = [start_date + timedelta(days=i) for i in range(delta.days + 1)]

# Erstellen einer Liste aller Feiertage von 2017 bis 2020
holiday_dates = []
for year in range(2017, 2021):
    holiday_dates.extend(get_holidays(year).values())

# Erstellen des Dictionary
data = {
    "date": [],
    "is_holiday": []
}

for day in dates:
    data["date"].append(day)
    data["is_holiday"].append(day in holiday_dates)

holiday_data = pd.DataFrame(data)

# Datentypen ändern
holiday_data['date'] = pd.to_datetime(holiday_data.date)

holiday_data.sample(5)

,date,is_holiday
391,2018-01-27,False
692,2018-11-24,False
113,2017-04-24,False
198,2017-07-18,False
1237,2020-05-22,False


## 5. Umfragedaten vorbereiten
Ignorierte Daten u.a.:
* Demographie Daten über Bewohner wie Ethie, Alter, Geschlecht, Bildungsabschluss, Einkommen
* Datails zu verwendeten Spalten, die nur in Text vorliegen und nicht direkt verwendbar sind wie Anzahl Stunden im Home Office, Veränderungsarbeiten am Haus (wie z.B. Reparatur, Austausch von Geräten, Neuinstallationen, Renovierungen, oft Anbringung von Solar Anlage)
* Details über Geräte wie Hersteller, typischer Stomverbrauch etc.

In [7]:
def convert_square_feet_to_square_meters(feet):
    """
    Funktion zur Umrechnung von Square Feet zu Quadratmeter (m²).
    :param foot: Fläche in Square Feet
    :return: Fläche in Quadratmeter (m²)
    """
    meters = round(feet * 0.092903, 2)
    return meters

In [8]:
survey_data = pd.read_csv('../input/2023-12-20/audits_surveys/survey_2013_all_participants.csv')

# Jahreszahlen einheitlich als Integer speichern
survey_data['construction_year'] = survey_data.year_house_constructed.replace('1930 or earlier', '1930').astype('Int64')

# Überführung der Werte aus Spalte primary_residence in Bool-Werte
# Leere Zeilen sorgen dafür, dass man nicht direkt in True/False konvertieren kann
survey_data['is_primary_residence'] = survey_data.primary_residence.map({'Yes': True, 'No': False}).astype('boolean')

# Berechnung der Anzahl der Bewohner über die Altersklassen
age_columns = ['residents_under_5', 'residents_6_to_12', 'residents_13_to_18', 'residents_19_to_24',
               'residents_25_to_34', 'residents_35_to_49', 'residents_50_to_64', 'residents_older_65']
survey_data['n_residents'] = survey_data[age_columns].sum(axis=1).astype('Int64')
survey_data = survey_data[survey_data.n_residents != 0] # Einträge mit 0 Bewohnern entfernen

# Datentypen ändern
survey_data['n_rooms'] = survey_data.house_num_rooms.astype('Int64')

# Umrechnung der Fläche von Square Feet zu Quadratmeter
survey_data['total_area'] = survey_data.house_square_feet.apply(convert_square_feet_to_square_meters)

# Auswahl der relevanten Spalten
relevant_columns = ['dataid', 'construction_year', 'is_primary_residence', 'n_residents', 'n_rooms', 'total_area']
survey_data = survey_data[relevant_columns]

survey_data.sample(5)

,dataid,construction_year,is_primary_residence,n_residents,n_rooms,total_area
301,8574,2012,True,3,3,130.06
139,4162,2005,True,6,12,371.61
90,9869,1986,True,3,6,183.95
238,6121,2010,True,4,3,204.39
243,59,2012,True,2,11,334.45


# 6. Datensatz auf ausgewählte Häuser beschränken

Auswahlkriterien:
1. Muss in survey 2013 aufgeführt sein, da hier die meisten verwendbaren Informationen gesammelt wurden
2. Top 10 Häuser mit den meisten Datenpunkten in 'Hauptdaten'

In [9]:
# Zählen der Vorkommen von dataid in main_data
counts = main_data.dataid.value_counts()

# Filtern basierend auf den dataids in survey_data
filtered_counts = counts[counts.index.isin(survey_data.dataid)]

# Top 10 dataids mit den meisten Vorkommen
top_10_dataids = filtered_counts.nlargest(10)

top_10_dataids

dataid
6139    35036
4031    35036
9922    35036
661     35032
7800    35020
4767    34959
3456    34932
5746    34736
1642    34648
Name: count, dtype: int64

## 7. Daten zusammenführen
Da zu den ausgewählten Häusern keine Daten zur Innentemperatur vorliegen, wird dieser Schritt übersprungen

In [10]:
main_data_filtered = main_data[main_data.dataid.isin(top_10_dataids.index)]

# Unlogische Werte aussortieren (könnte z.B. auf Stromausfall oder fehlerhafte Messung hindeuten)
main_data_filtered = main_data_filtered[main_data_filtered.use != 0]

# Stündlichen Durchschnitt für Werte bilden
main_data_filtered.set_index('timestamp', inplace=True)
main_data_hourly = main_data_filtered.groupby(['dataid', pd.Grouper(freq='H')]).mean().reset_index()

# Umfragedaten joinen
df = pd.merge(main_data_hourly, survey_data, on='dataid', how='left')

# Wetterdaten joinen
df = pd.merge(df, weather_data, on='timestamp', how='left')

# Feiertagsdaten joinen
df['date'] = pd.to_datetime(df.timestamp.dt.date)
df = pd.merge(df, holiday_data, on='date', how='left')
df.drop(columns=['date'], inplace=True)

# Entfernen von leeren Spalten
df.dropna(axis=1, how='all', inplace=True)

df.sample(5)

,dataid,timestamp,air1,air2,airwindowunit1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,...,use,construction_year,is_primary_residence,n_residents,n_rooms,total_area,outdoor_temp,app_outdoor_temp,humidity,is_holiday
87049,9922,2018-09-02 10:00:00,0.97400,-0.00525,NaN,NaN,NaN,0.02975,0.10850,0.06975,...,2.23200,2006,True,2,5,213.68,28.79,32.27,0.70,False
19269,3456,2018-02-26 21:00:00,0.00100,NaN,NaN,0.01675,NaN,0.00550,NaN,NaN,...,0.78275,2008,True,2,10,159.79,15.19,15.19,0.78,False
31621,4031,2018-04-08 13:00:00,NaN,NaN,NaN,NaN,NaN,0.02575,NaN,NaN,...,0.52800,2008,True,3,10,190.45,12.09,12.09,0.70,False
89139,9922,2018-11-28 12:00:00,0.00125,-0.00475,NaN,NaN,NaN,0.11150,0.09875,0.00950,...,1.24850,2006,True,2,5,213.68,18.44,18.44,0.54,False
414,661,2018-01-18 13:00:00,0.00000,NaN,NaN,0.00450,NaN,NaN,NaN,NaN,...,0.64625,2007,True,2,6,154.59,3.31,1.37,0.39,False


## 8. Features ergänzen

In [11]:
df['day'] = df.timestamp.dt.day
df['wday'] = df.timestamp.dt.dayofweek  # 0 = Montag, 6 = Sonntag
df['month'] = df.timestamp.dt.month
df['year'] = df.timestamp.dt.year
df['hour'] = df.timestamp.dt.hour

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89949 entries, 0 to 89948
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   dataid                89949 non-null  int64         
 1   timestamp             89949 non-null  datetime64[ns]
 2   air1                  71807 non-null  float64       
 3   air2                  8695 non-null   float64       
 4   airwindowunit1        8993 non-null   float64       
 5   bathroom1             26461 non-null  float64       
 6   bathroom2             8671 non-null   float64       
 7   bedroom1              44911 non-null  float64       
 8   bedroom2              17680 non-null  float64       
 9   bedroom3              8695 non-null   float64       
 10  car1                  36019 non-null  float64       
 11  clotheswasher1        80369 non-null  float64       
 12  diningroom1           9049 non-null   float64       
 13  dishwasher1     

In [12]:
# Als csv-Datei speichern
df.to_csv('austin10_dataset.csv', index=False)